In [9]:
!cp po5.db po7.db

In [10]:
'''
This file to edit the tables in the database
I have alreaady edited field_book where I dropped the duplicates and set the plant_name to be the primary key
'''

'\nThis file to edit the tables in the database\nI have alreaady edited field_book where I dropped the duplicates and set the plant_name to be the primary key\n'

In [11]:
'''
In the following I will edit rgb table
I will drop all the non relevant columns like:
    treatment
    plot 
    genotype 
I will keep the plant_name so it can be a foriegn key to field_book
That way we can lose redundant data and we can get this data from the 
field_book.

One conclusion I got from field book:
      {treatment, plot, genotype} hasn't changed overtime for each scan
'''
import sqlite3
import pandas as pd
from IPython.display import display

# global values
po5 = sqlite3.connect('po5.db')
po7 = sqlite3.connect('po7.db')
rgb = pd.read_sql_query('SELECT * FROM rgb', po5)


In [12]:
'''
ops:
1) drop columns
2) set primary keys
'''
#1) drop columns
drop_unwanted_columns = rgb.drop(columns=['level_0','index','treatment', 'plot', 'genotype'])
display(drop_unwanted_columns)


,date,plant_name,lon,lat,min_x,max_x,min_y,max_y,nw_lat,nw_lon,se_lat,se_lon,bounding_area_m2,double_plant,partial_scan,flags
0,2019-12-10,Green_Thunder_72,-111.974954,33.076240,603.0,752.0,6884.0,7030.0,33.076240,-111.974954,33.076241,-111.974953,0.002087,0.0,0,"['no_dp', 'no_ps']"
1,2019-12-10,Green_Thunder_179,-111.974954,33.076231,557.0,729.0,10105.0,10314.0,33.076231,-111.974954,33.076232,-111.974954,0.003463,0.0,0,"['no_dp', 'no_ps']"
2,2019-12-10,Green_Thunder_26,-111.974954,33.076233,556.0,729.0,9688.0,9813.0,33.076232,-111.974954,33.076233,-111.974954,0.002063,0.0,0,"['no_dp', 'no_ps']"
3,2019-12-10,Green_Thunder_27,-111.974953,33.076257,670.0,851.0,892.0,1098.0,33.076257,-111.974954,33.076258,-111.974953,0.003587,0.0,0,"['no_dp', 'no_ps']"
4,2019-12-10,Green_Thunder_111,-111.974954,33.076234,566.0,693.0,9060.0,9210.0,33.076234,-111.974954,33.076235,-111.974954,0.001834,0.0,0,"['no_dp', 'no_ps']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522425,2020-03-03,Infantry_44,-111.975026,33.075678,38.0,719.0,840.0,1673.0,33.075676,-111.975028,33.075679,-111.975025,0.062960,0.0,0,"['no_dp', 'no_ps']"
522426,2020-03-03,Infantry_116,-111.975026,33.075657,24.0,658.0,7555.0,8466.0,33.075656,-111.975028,33.075658,-111.975025,0.064303,0.0,0,"['no_dp', 'no_ps']"
522427,2020-03-03,Infantry_221,-111.975026,33.075653,25.0,682.0,9004.0,9634.0,33.075652,-111.975028,33.075654,-111.975025,0.045730,0.0,0,"['no_dp', 'no_ps']"
522428,2020-03-03,Infantry_13,-111.975026,33.075650,0.0,719.0,10112.0,10901.0,33.075648,-111.975028,33.075651,-111.975025,0.062833,1.0,0,"['dp', 'no_ps']"


In [13]:
'''
From the data we can see tha each plant has one scan per date
let's check that out
'''
#check that date+name is not redundant
name_date = drop_unwanted_columns['plant_name'] + drop_unwanted_columns['date']
print(f'len(name_date): {len(name_date)}')
print(f'len(set(name_date)): {len(set(name_date))}')
if(len(name_date) == len(set(name_date))):
    print('equal')
#Thus, the combination of date name is unique

len(name_date): 522430
len(set(name_date)): 522430
equal


In [14]:
'''
commit new table drop_unwanted_columns
then edit with the script in the next cell
'''
schema = '''
CREATE TABLE IF NOT EXISTS "rgb" (
"level_0" INTEGER,
  "index" INTEGER,
  "date" TEXT NO NULL,
  "plant_name" TEXT NO NULL,
  "lon" REAL,
  "lat" REAL,
  "min_x" REAL,
  "max_x" REAL,
  "min_y" REAL,
  "max_y" REAL,
  "nw_lat" REAL,
  "nw_lon" REAL,
  "se_lat" REAL,
  "se_lon" REAL,
  "bounding_area_m2" REAL,
  "double_plant" REAL,
  "partial_scan" INTEGER,
  "flags" TEXT,
  PRIMARY KEY (plant_name, date)
);
'''
drop_unwanted_columns.to_sql('rgb', po7, if_exists='replace')

522430

In [15]:
sql_command = '''

PRAGMA foreign_keys=off;

BEGIN TRANSACTION;

ALTER TABLE rgb RENAME TO old_rgb;

-- define the primary key constraint here
CREATE TABLE IF NOT EXISTS "rgb" (
"index" INTEGER,
  "date" TEXT NO NULL,
  "plant_name" TEXT NO NULL,
  "lon" REAL,
  "lat" REAL,
  "min_x" REAL,
  "max_x" REAL,
  "min_y" REAL,
  "max_y" REAL,
  "nw_lat" REAL,
  "nw_lon" REAL,
  "se_lat" REAL,
  "se_lon" REAL,
  "bounding_area_m2" REAL,
  "double_plant" REAL,
  "partial_scan" INTEGER,
  "flags" TEXT,
  PRIMARY KEY ( "plant_name", "date")
);

INSERT INTO rgb SELECT * FROM old_rgb;

DROP TABLE old_rgb;

COMMIT;

PRAGMA foreign_keys=on;


'''
cursor = po7.cursor()
cursor.executescript(sql_command)